In [2]:
import os
import csv
import pandas as pd
import numpy as np
import glob
import sqlite3

### Import CSVs and create concatenated df for single stocks


In [ ]:
# Define working directory
os.chdir(r'C:\Users\muell\Desktop\Thesis\MA\Data')

# Get names of all stocks through the folder structure 
stocklist = os.listdir()

# Loop to create and store (in SQL) all tweet dataframes/tables, segmented by stock ticker
for a in range(0,len(stocklist)):
    
    # Define stock ticker for respective position nr in folder
    stock = stocklist[a]

    # Define path where csv files can be found
    path = os.getcwd() + '\\' + stocklist[a] + '\\'

    # Get filenames created through using multiple threads when sourcing tweets to concatenate files together later on
    # Gets list of all files in respective path folder
    filelist = os.listdir(path) 
    # Creates list of all files
    all_files = glob.glob(os.path.join(path, "*.csv")) 

    # Imports all files as df, during import 0.45% to 1.64% of obs are lost due to wrong format in csv input
    df_from_each_file = (pd.read_csv(f, sep=',',   lineterminator='\n',\
                                     error_bad_lines=False, warn_bad_lines=False, quoting=csv.QUOTE_NONE) for f in all_files) 

    # Concatenates all df's of respective Ticker to one df
    concatenated_df  = pd.concat(df_from_each_file, ignore_index=True, sort=False)

    # Cut of excessive columns, which are added due to somehow non-dispenseable but automatic semicolons in csv file
    df_size = 26
    for i in range(df_size,len(concatenated_df.columns)):
        if len(concatenated_df.columns)>df_size:
            concatenated_df = concatenated_df.drop([concatenated_df.columns[df_size]], axis='columns')

    # Remove all columns that are not required in the analysis
    for i in [26,25,24,23,22,21,20,12,11,10,9,8,7,5,4,3,2,1,6]:
        try:
            concatenated_df = concatenated_df.drop([concatenated_df.columns[i]], axis='columns')
        except: 
            continue

    # Drop all rows which have nan in the last two columns, as a proxy for files with wrong data structure        
    concatenated_df = concatenated_df.drop(concatenated_df.index[concatenated_df["is Retweet?"].isnull() == True].tolist())
    concatenated_df = concatenated_df.drop(concatenated_df.index[concatenated_df["is Reply?"].isnull() == True].tolist())

    # Change format of date to datetime
    concatenated_df['Date']= pd.to_datetime(concatenated_df['Date']) 

    # Create connection to SQL DB, or if done for the first time: create db
    conn = sqlite3.connect(r"C:\Users\muell\Desktop\Thesis\MA\SQL_DB\Tw_DB.db")
    # Save the df on SQL DB
    concatenated_df.to_sql('{}'.format(stock), conn, if_exists='replace', index=False)
    # Close connection to SQL DB
    conn.close()

### Data Pre-Processing

In [ ]:
import timeit


os.chdir(r'C:\Users\muell\Desktop\Thesis\MA\Data')
stocklist = os.listdir()


# Loop to create and store (in SQL) all tweet dataframes/tables, segmented by stock ticker
# for a in range(0,len(stocklist)):

 
for a in reversed(range(0,len(stocklist))):

    start_time = timeit.default_timer()


    stock_adj = stocklist[a] + '_adj'
    stock = stocklist[a]

    conn = sqlite3.connect(r"C:\Users\muell\Desktop\Thesis\MA\SQL_DB\Tw_DB.db")
    c = conn.cursor()
    # Select entire table of respective stock ticker
    query = '''select * from {} '''.format(stock)
    c.execute(query)
    # Convert to pandas DF format
    result = pd.DataFrame(c.fetchall())
    # Rename all columns accordingly
    result.rename(columns={0: 'ID', 1: 'Date', 2: 'Text', 3: 'N_Replies', 4: 'N_Retweets', 5: 'N_Favorites', 6: 'D_Reply', 7: 'D_Retweet'}, inplace=True)
    # Change format of Date from object to datetime
    # result['Date']= pd.to_datetime(result['Date']) 
    # Drop non-essential columns for now
    result = result[['ID','Text']]
    total_rows = result.Text.count()
    if total_rows < 60000:
        print('0. ' + str(timeit.default_timer() - start_time))

        # Converts all letters into lower case
        result['Text'] = result['Text'].str.lower()

        import re
        # Delete all http & https URLs
        result['Text'] = [re.sub(r"http\S+|rhttps\S+|rwww\S+", "", e) for e in result['Text']]

        # Delete all User Mentions
        result['Text'] = [re.sub(r"@\S+", "", e) for e in result['Text']]

        # Remove all hashtags
        result['Text'] = [re.sub(r'#([^\s]+)', r'\1', e) for e in result['Text']]

        # Count number of $ in each Tweet to use as a proxy for tweet specificity (i.e. number of 
        # stock tickers referenced in each tweet) later on
        result['N_Ticker'] = [e.count("$") for e in result['Text']]

        # Remove all Cashtags 
        result['Text'] = [re.sub(r"\$\S+", '', e) for e in result['Text']]

        # Mitigate elongated words, by detecting positions where character is repeated 3+ times and reduced to 1 time (approximation)
        result['Text'] = [re.sub(r'((\w)\2{2,})', r'\2', e) for e in result['Text']]
        # Apply spellchecker to correct elongated words (and increase performance of above approximation)
        # Extremely time intense (ignore for now)
        # from autocorrect import Speller
        # check = Speller(lang='en')
        print('1. ' + str(timeit.default_timer() - start_time))

        import nltk
        # list of english words, used to later remove non-english words
        words = set(nltk.corpus.words.words())

        # Remove all stopwords from the tweets
        import nltk
        # nltk.download('punkt')
        from nltk.corpus import stopwords
        # nltk.download('stopwords')
        from nltk.tokenize import word_tokenize
        # Tokenize the respective tweets
        tweet_tokens = [word_tokenize(e) for e in result['Text']]
        print('2. ' + str(timeit.default_timer() - start_time))
        # Define the list of stopwords which will be removed
        all_stopwords = stopwords.words('english')
        #remove_list = []
        # all_stopwords = [word for word in all_stopwords if word not in remove_list]
        # Add additional stop words to be removed from texts
        sw_list = ['amp', 'aint', "ain't", 'barely', 'cannot ', 'cant', "can't", 'couldnt', "couldn't", 'darent', "daren't", 'didnt', \
        "didn't", "doesnt", "doesn't", 'dont', "don't", 'hadnt', "hadn't", 'hardly', 'hasnt', "hasn't", 'havent', \
        "haven't", 'isnt', "isn't", 'lack', 'lacking', 'lacks', 'mightnt', "mightn't", 'mustnt', "mustn't", \
        'neednt', "needn't", 'neither', 'never', 'no one', 'nobody', 'none', 'nor',  'nothing', \
        'nowhere', 'oughtnt', "oughtn't", 'scarcely', 'shant', "shan't", 'shouldnt', "shouldn't", 'wasnt', \
        "wasn't", 'without', 'wont', "won't", 'wouldnt', "wouldn't"]
        all_stopwords.extend(sw_list)


        # Lemmatize the words ( preferred relative to stemming )
        from nltk.stem import WordNetLemmatizer
        lemmatizer = WordNetLemmatizer()
        # Tokenize the respective tweets
        tweet_tokens = [word_tokenize(e) for e in result['Text']]
        print('3. ' + str(timeit.default_timer() - start_time))
        for i in range(0,len(tweet_tokens)):
            for p in ['a','s','r','n','v']:
                tweet_tokens[i] = [lemmatizer.lemmatize(w, pos=p) for w in tweet_tokens[i]] 
            # Delete all stopwords
            tweet_tokens[i] = [word for word in tweet_tokens[i] if word not in all_stopwords]

                # join list of words back to a sentence    
            # result.loc[i, 'Text'] = ' '.join(tweet_tokens[i])
            # Delete all words which are non-english to reduce dimensionality of matrix later on
            result.loc[i, 'Text'] = [" ".join(w for w in nltk.wordpunct_tokenize(str(tweet_tokens[i])) if w in words or not w.isalpha())]
        print('4. ' + str(timeit.default_timer() - start_time))

        # Negation handling:
        # Detects negations and transforms negated words into 'not_' form
        def negate_sequence(text):
            negation = False
            delims = "?.,!:;"
            result = []
            words = text.split()
            prev = None
            pprev = None
            for word in words:
                stripped = word.strip(delims).lower()
                negated = "not_" + stripped if negation else stripped
                result.append(negated)
                prev = negated

                if any(neg in word for neg in ['aint', "ain't", 'arent', "aren't" 'barely', 'cannot ', 'cant', "can't", 'couldnt', "couldn't", 'darent', "daren't", 'didnt', \
                                                "didn't", "doesnt", "doesn't", 'dont', "don't", 'hadnt', "hadn't", 'hardly', 'hasnt', "hasn't", 'havent', \
                                                "haven't", 'isnt', "isn't", 'lack', 'lacking', 'lacks', 'mightnt', "mightn't", 'mustnt', "mustn't", \
                                                'neednt', "needn't", 'neither', 'never', 'no', 'no one', 'nobody', 'none', 'nor', 'not', 'nothing', \
                                                'nowhere', 'oughtnt', "oughtn't", 'scarcely', 'shant', "shan't", 'shouldnt', "shouldn't", 'wasnt', \
                                                "wasn't", 'werent', "weren't", 'without', 'wont', "won't", 'wouldnt', "wouldn't"]):
                    negation = not negation

                if any(c in word for c in delims):
                    negation = False

            return result
        # Application of def negate_sequence for negation handling
        result['Text'] = [' '.join(negate_sequence(e)) for e in result['Text']]
        print('5. ' + str(timeit.default_timer() - start_time))



        # Remove all punctuations and all special chars and replace with space
        import string
        result['Text'] = result['Text'].str.replace('[^\w\s]',' ')

        # Remove all numbers
        result['Text'] = [re.sub("\d+", "", e) for e in result['Text']]

        # Remove short words with 2 chars or less
        result['Text'] = [re.sub(r'\b\w{1,2}\b', '', e) for e in result['Text']]

        # Replace multiple whitespaces with single whitespace
        result['Text'] = [' '.join(str(e).split()) for e in result['Text']]
        print('6. ' + str(timeit.default_timer() - start_time))

        # Save the new _adj df on SQL DB
        result.to_sql('{}'.format(stock_adj), conn, if_exists='replace', index=False)
        # Close connection to SQL DB
        conn.close()
        # code you want to evaluate
        print('7. ' + str(timeit.default_timer() - start_time))
        print(stock_adj)
        print(result)
        # result.to_csv(r'C:\Users\muell\Desktop\Thesis\MA\Test\WBA.csv')

    else:
        # 20000 is the optimal size for efficiency reasons
        N_splits = int(round(total_rows/20000))
        result_arr = np.array_split(result,N_splits)
        
        def large_pre_proc(arr_num):
            result = result_arr[arr_num]
            
            # to account for shift in loop below
            sum_shift = 0
            for ab in range(0,arr_num):
                sum_shift = len(result_arr[ab]) + sum_shift

            print('0. ' + str(timeit.default_timer() - start_time))

            # Converts all letters into lower case
            result['Text'] = result['Text'].str.lower()

            import re
            # Delete all http & https URLs
            result['Text'] = [re.sub(r"http\S+|rhttps\S+|rwww\S+", "", e) for e in result['Text']]

            # Delete all User Mentions
            result['Text'] = [re.sub(r"@\S+", "", e) for e in result['Text']]

            # Remove all hashtags
            result['Text'] = [re.sub(r'#([^\s]+)', r'\1', e) for e in result['Text']]

            # Count number of $ in each Tweet to use as a proxy for tweet specificity (i.e. number of 
            # stock tickers referenced in each tweet) later on
            result['N_Ticker'] = [int(e.count("$")) for e in result['Text']]

            # Remove all Cashtags 
            result['Text'] = [re.sub(r"\$\S+", '', e) for e in result['Text']]

            # Mitigate elongated words, by detecting positions where character is repeated 3+ times and reduced to 1 time (approximation)
            result['Text'] = [re.sub(r'((\w)\2{2,})', r'\2', e) for e in result['Text']]
            # Apply spellchecker to correct elongated words (and increase performance of above approximation)
            # Extremely time intense (ignore for now)
            # from autocorrect import Speller
            # check = Speller(lang='en')
            print('1. ' + str(timeit.default_timer() - start_time))

            import nltk
            # list of english words, used to later remove non-english words
            words = set(nltk.corpus.words.words())

                       # Remove all stopwords from the tweets
            import nltk
            # nltk.download('punkt')
            from nltk.corpus import stopwords
            # nltk.download('stopwords')
            print('2. ' + str(timeit.default_timer() - start_time))
            # Define the list of stopwords which will be removed
            all_stopwords = stopwords.words('english')
            #remove_list = []
            # all_stopwords = [word for word in all_stopwords if word not in remove_list]
            # Add additional stop words to be removed from texts
            sw_list = ['amp', 'aint', "ain't", 'barely', 'cannot ', 'cant', "can't", 'couldnt', "couldn't", 'darent', "daren't", 'didnt', \
            "didn't", "doesnt", "doesn't", 'dont', "don't", 'hadnt', "hadn't", 'hardly', 'hasnt', "hasn't", 'havent', \
            "haven't", 'isnt', "isn't", 'lack', 'lacking', 'lacks', 'mightnt', "mightn't", 'mustnt', "mustn't", \
            'neednt', "needn't", 'neither', 'never', 'no one', 'nobody', 'none', 'nor',  'nothing', \
            'nowhere', 'oughtnt', "oughtn't", 'scarcely', 'shant', "shan't", 'shouldnt', "shouldn't", 'wasnt', \
            "wasn't", 'without', 'wont', "won't", 'wouldnt', "wouldn't"]
            all_stopwords.extend(sw_list)


            # Lemmatize the words ( preferred relative to stemming )
            from nltk.stem import WordNetLemmatizer
            lemmatizer = WordNetLemmatizer()
            # Tokenize the respective tweets
            from nltk.tokenize import word_tokenize
            tweet_tokens = [word_tokenize(e) for e in result['Text']]
            print('3. ' + str(timeit.default_timer() - start_time))
            for i in range(0,len(tweet_tokens)):
                for p in ['a','s','r','n','v']:
                    tweet_tokens[i] = [lemmatizer.lemmatize(w, pos=p) for w in tweet_tokens[i]] 
                # Delete all stopwords
                tweet_tokens[i] = [word for word in tweet_tokens[i] if word not in all_stopwords]

                    # join list of words back to a sentence    
                # result.loc[i, 'Text'] = ' '.join(tweet_tokens[i])
                # Delete all words which are non-english to reduce dimensionality of matrix later on
                result.loc[(i+sum_shift), 'Text'] = [" ".join(w for w in nltk.wordpunct_tokenize(str(tweet_tokens[i])) if w in words or not w.isalpha())]
            print('4. ' + str(timeit.default_timer() - start_time))

            # Negation handling:
            # Detects negations and transforms negated words into 'not_' form
            def negate_sequence(text):
                negation = False
                delims = "?.,!:;"
                result = []
                words = text.split()
                prev = None
                pprev = None
                for word in words:
                    stripped = word.strip(delims).lower()
                    negated = "not_" + stripped if negation else stripped
                    result.append(negated)
                    prev = negated

                    if any(neg in word for neg in ['aint', "ain't", 'arent', "aren't" 'barely', 'cannot ', 'cant', "can't", 'couldnt', "couldn't", 'darent', "daren't", 'didnt', \
                                                    "didn't", "doesnt", "doesn't", 'dont', "don't", 'hadnt', "hadn't", 'hardly', 'hasnt', "hasn't", 'havent', \
                                                    "haven't", 'isnt', "isn't", 'lack', 'lacking', 'lacks', 'mightnt', "mightn't", 'mustnt', "mustn't", \
                                                    'neednt', "needn't", 'neither', 'never', 'no', 'no one', 'nobody', 'none', 'nor', 'not', 'nothing', \
                                                    'nowhere', 'oughtnt', "oughtn't", 'scarcely', 'shant', "shan't", 'shouldnt', "shouldn't", 'wasnt', \
                                                    "wasn't", 'werent', "weren't", 'without', 'wont', "won't", 'wouldnt', "wouldn't"]):
                        negation = not negation

                    if any(c in word for c in delims):
                        negation = False

                return result
            # Application of def negate_sequence for negation handling
            result['Text'] = [' '.join(negate_sequence(e)) for e in result['Text']]
            print('5. ' + str(timeit.default_timer() - start_time))



            # Remove all punctuations and all special chars and replace with space
            import string
            result['Text'] = result['Text'].str.replace('[^\w\s]',' ')

            # Remove all numbers
            result['Text'] = [re.sub("\d+", "", e) for e in result['Text']]

            # Remove short words with 2 chars or less
            result['Text'] = [re.sub(r'\b\w{1,2}\b', '', e) for e in result['Text']]

            # Replace multiple whitespaces with single whitespace
            result['Text'] = [' '.join(str(e).split()) for e in result['Text']]
            print('6. ' + str(timeit.default_timer() - start_time))

            return result

        final_result = list(map(large_pre_proc,list(range(0, (N_splits)))))
        
        # Merge file
        new_final_result = pd.DataFrame(final_result[0])
        for x in range (1,N_splits):
            new_final_result = new_final_result.append(pd.DataFrame(final_result[x]), ignore_index=True) 
        print(new_final_result)
        # Save the new _adj df on SQL DB
        new_final_result.to_sql('{}'.format(stock_adj), conn, if_exists='replace', index=False)
        # Close connection to SQL DB
        conn.close()
        # code you want to evaluate
        print('7. ' + str(timeit.default_timer() - start_time))
        print(stock_adj)

        # result.to_csv(r'C:\Users\muell\Desktop\Thesis\MA\Test\WBA.csv')

### Retry with better negation handling

In [172]:
import timeit


os.chdir(r'C:\Users\muell\Desktop\Thesis\MA\Data')
stocklist = os.listdir()


# Loop to create and store (in SQL) all tweet dataframes/tables, segmented by stock ticker
# for a in range(0,len(stocklist)):

 
for a in reversed(range(0,len(stocklist))):

    start_time = timeit.default_timer()


    stock_adj = stocklist[a] + '_adj'
    stock = stocklist[a]

    conn = sqlite3.connect(r"C:\Users\muell\Desktop\Thesis\MA\SQL_DB\Tw_DB.db")
    c = conn.cursor()
    # Select entire table of respective stock ticker
    query = '''select * from {} '''.format(stock)
    c.execute(query)
    # Convert to pandas DF format
    result = pd.DataFrame(c.fetchall())
    # Rename all columns accordingly
    result.rename(columns={0: 'ID', 1: 'Date', 2: 'Text', 3: 'N_Replies', 4: 'N_Retweets', 5: 'N_Favorites', 6: 'D_Reply', 7: 'D_Retweet'}, inplace=True)
    # Change format of Date from object to datetime
    # result['Date']= pd.to_datetime(result['Date']) 
    # Drop non-essential columns for now
    result = result[['ID','Text']]
    total_rows = result.Text.count()
    # 20000 is the optimal size for efficiency reasons
    N_splits = int(round(total_rows/20000))
    result_arr = np.array_split(result,N_splits)

    def large_pre_proc(arr_num):
        result = result_arr[arr_num]

        # to account for shift in loop below
        sum_shift = 0
        for ab in range(0,arr_num):
            sum_shift = len(result_arr[ab]) + sum_shift
        print('0. ' + str(timeit.default_timer() - start_time))

        # Converts all letters into lower case
        result['Text'] = result['Text'].str.lower()

        import re
        # Delete all http & https URLs
        result['Text'] = [re.sub(r"http\S+|rhttps\S+|rwww\S+", "", e) for e in result['Text']]

        # Delete all User Mentions
        result['Text'] = [re.sub(r"@\S+", "", e) for e in result['Text']]

        # Remove all hashtags
        result['Text'] = [re.sub(r'#([^\s]+)', r'\1', e) for e in result['Text']]

        # Count number of $ in each Tweet to use as a proxy for tweet specificity (i.e. number of 
        # stock tickers referenced in each tweet) later on
        result['N_Ticker'] = [int(e.count("$")) for e in result['Text']]

        # Remove all Cashtags 
        result['Text'] = [re.sub(r"\$\S+", '', e) for e in result['Text']]

        # Mitigate elongated words, by detecting positions where character is repeated 3+ times and reduced to 1 time (approximation)
        result['Text'] = [re.sub(r'((\w)\2{2,})', r'\2', e) for e in result['Text']]
        # Apply spellchecker to correct elongated words (and increase performance of above approximation)
        # Extremely time intense (ignore for now)
        # from autocorrect import Speller
        # check = Speller(lang='en')
        print('1. ' + str(timeit.default_timer() - start_time))

        import nltk
        # list of english words, used to later remove non-english words
        words = set(nltk.corpus.words.words())

        # nltk.download('punkt')
        from nltk.corpus import stopwords
        # nltk.download('stopwords')
        from nltk.tokenize import word_tokenize

        # Lemmatize the words ( preferred relative to stemming )
        from nltk.stem import WordNetLemmatizer
        lemmatizer = WordNetLemmatizer()
        # Tokenize the respective tweets
        tweet_tokens = [word_tokenize(e) for e in result['Text']]
        print('3. ' + str(timeit.default_timer() - start_time))
        for i in range(0,len(tweet_tokens)):
            for p in ['a','s','r','n','v']:
                tweet_tokens[i] = [lemmatizer.lemmatize(w, pos=p) for w in tweet_tokens[i]] 
                # join list of words back to a sentence    
            # result.loc[i, 'Text'] = ' '.join(tweet_tokens[i])
            # Delete all words which are non-english to reduce dimensionality of matrix later on
            result.loc[(i+sum_shift), 'Text'] = [" ".join(w for w in nltk.wordpunct_tokenize(str(tweet_tokens[i])) if w in words or not w.isalpha())]
        print('4. ' + str(timeit.default_timer() - start_time))

        # Negation handling:
        # Detects negations and transforms negated words into 'not_' form
        def negate_sequence(text):
            negation = False
            delims = "?.,!:;"
            result = []
            words = text.split()
            prev = None
            pprev = None
            for word in words:
                stripped = word.strip(delims).lower()
                negated = "not_" + stripped if negation else stripped
                result.append(negated)
                prev = negated

                if any(neg in word for neg in [ 'barely', 'cannot',  'hardly', 'lack', 'lacking', 'lacks', 'neither', 'never', \
                                               'no one', 'nobody', 'none',  'nothing', 'nowhere',  'scarcely', \
                                               'without', ' not ', "n't", " no " ]):
                    negation = not negation

                if any(c in word for c in delims):
                    negation = False

            return result
        # Application of def negate_sequence for negation handling
        result_temp = result['Text'].apply(negate_sequence)            
        result["Text"] = list(" ".join(e) for e in result_temp)
        print('5. ' + str(timeit.default_timer() - start_time))
        # Remove stopwords
        # Tokenize the respective tweets
        tweet_tokens = [word_tokenize(e) for e in result['Text']]
        print('6. ' + str(timeit.default_timer() - start_time))
        # Define the list of stopwords which will be removed
        all_stopwords = stopwords.words('english')
        #remove_list = []
        # all_stopwords = [word for word in all_stopwords if word not in remove_list]
        # Add additional stop words to be removed from texts
        sw_list = ['amp', 'aint', "ain't", 'barely', 'cannot ', 'cant', "can't", 'couldnt', "couldn't", 'darent', "daren't", 'didnt', \
        "didn't", "doesnt", "doesn't", 'dont', "don't", 'hadnt', "hadn't", 'hardly', 'hasnt', "hasn't", 'havent', \
        "haven't", 'isnt', "isn't", 'lack', 'lacking', 'lacks', 'mightnt', "mightn't", 'mustnt', "mustn't", \
        'neednt', "needn't", 'neither', 'never', 'no one', 'nobody', 'none', 'nor',  'nothing', \
        'nowhere', 'oughtnt', "oughtn't", 'scarcely', 'shant', "shan't", 'shouldnt', "shouldn't", 'wasnt', \
        "wasn't", 'without', 'wont', "won't", 'wouldnt', "wouldn't"]
        all_stopwords.extend(sw_list)
#        remove_list = ['not']
#        all_stopwords = [word for word in all_stopwords if word not in remove_list]
        for i in range(0,len(tweet_tokens)):        
            # Delete all stopwords
            result.loc[(i+sum_shift), 'Text'] = [" ".join(word for word in tweet_tokens[i] if word not in all_stopwords)]        

        # Remove all punctuations and all special chars and replace with space
        import string
        result['Text'] = result['Text'].str.replace('[^\w\s]',' ')

        # Remove all numbers
        result['Text'] = [re.sub("\d+", "", e) for e in result['Text']]

        # Remove short words with 2 chars or less
        result['Text'] = [re.sub(r'\b\w{1,2}\b', '', e) for e in result['Text']]

        # Replace multiple whitespaces with single whitespace
        result['Text'] = [' '.join(str(e).split()) for e in result['Text']]
        print('7. ' + str(timeit.default_timer() - start_time))

        return result

    final_result = list(map(large_pre_proc,list(range(0, (N_splits)))))

    # Merge file
    new_final_result = pd.DataFrame(final_result[0])
    for x in range (1,(N_splits)):
        new_final_result = new_final_result.append(pd.DataFrame(final_result[x]), ignore_index=True) 
    print(new_final_result)
    # Save the new _adj df on SQL DB
    new_final_result.to_sql('{}'.format(stock_adj), conn, if_exists='replace', index=False)
    # Close connection to SQL DB
    conn.close()
    # code you want to evaluate
    print('8. ' + str(timeit.default_timer() - start_time))
    print(stock_adj)

# new_final_result.to_csv(r'C:\Users\muell\Desktop\Thesis\MA\Test\WBA.csv')

0. 0.2031680000363849
1. 0.6521559000248089
3. 3.547253600030672
4. 24.208250700030476
5. 26.513613100047223
[' short ' ' sale ' ' volume ' '(' ' not not_' ' short ' ' interest ' ')' ' for ' ' at ' ' 2019 - 12 - 27 ' ' be ' ' 44 ' '%' '.' ' 48 ' '%' ' 72 ' '%' ' 36 ' '%' ' 51 ' '%']
6. 30.16239690000657
7. 48.0102034000447
short sale volume not_ short interest
0. 48.05067650001729
1. 48.36541590001434
3. 50.79544120002538
4. 72.69716740003787
5. 74.53657140000723
[' ' ' group ' ' ' ' sell ' ' 49 ' ';' ' 263 ' ' share ' ' of ' ' ' ' instrument ' ' incorporate ']
6. 77.64345710002817
7. 95.72364490001928
group sell share instrument incorporate
                        ID                                               Text  \
0      1212153219473784833  plan head refuse enter stop loss order year ag...   
1      1212138180738256897                               best nasdaq heat map   
2      1212116817583337473  day move average cross day move average view o...   
3      1212116312597630976

### Pre-Processing  annotated Tweets (for lexicon extension)

In [33]:
import timeit
start_time = timeit.default_timer()
# Import annotated tweets dataset
columns =  ["Sentiment", "ID", "Date", "Flag", "User", "Text"]
encoding = "ISO-8859-1"
tagg_tweets = pd.read_csv(r'C:\Users\muell\Desktop\Thesis\MA\Data_Tagged\training_1600000.csv', \
                          encoding=encoding , names=columns)

# Only keep relevant data
tagg_tweets = tagg_tweets[['Sentiment','Text']]

# Replace positive sentiment dummy (i.e. =4) with 1. Negative sentiment = 0
tagg_tweets['Sentiment'] = tagg_tweets['Sentiment'].replace(4,1)

# Go through same pre-processing as above
total_rows = tagg_tweets.Text.count()
N_splits = int(round(total_rows/20000))
tagg_tweets_arr = np.array_split(tagg_tweets,N_splits)

def large_pre_proc(arr_num):
    tagg_tweets = tagg_tweets_arr[arr_num]
    # to account for shift in loop below
    sum_shift = 0
    for ab in range(0,arr_num):
        sum_shift = len(tagg_tweets_arr[ab]) + sum_shift

    print('0. ' + str(timeit.default_timer() - start_time))

    # Converts all letters into lower case
    tagg_tweets['Text'] = tagg_tweets['Text'].str.lower()

    import re
    # Delete all http & https URLs
    tagg_tweets['Text'] = [re.sub(r"http\S+|rhttps\S+|rwww\S+", "", e) for e in tagg_tweets['Text']]

    # Delete all User Mentions
    tagg_tweets['Text'] = [re.sub(r"@\S+", "", e) for e in tagg_tweets['Text']]

    # Remove all hashtags
    tagg_tweets['Text'] = [re.sub(r'#([^\s]+)', r'\1', e) for e in tagg_tweets['Text']]

    # Count number of $ in each Tweet to use as a proxy for tweet specificity (i.e. number of 
    # stock tickers referenced in each tweet) later on
    tagg_tweets['N_Ticker'] = [int(e.count("$")) for e in tagg_tweets['Text']]

    # Remove all Cashtags 
    tagg_tweets['Text'] = [re.sub(r"\$\S+", '', e) for e in tagg_tweets['Text']]

    # Mitigate elongated words, by detecting positions where character is repeated 3+ times and reduced to 1 time (approximation)
    tagg_tweets['Text'] = [re.sub(r'((\w)\2{2,})', r'\2', e) for e in tagg_tweets['Text']]
    # Apply spellchecker to correct elongated words (and increase performance of above approximation)
    # Extremely time intense (ignore for now)
    # from autocorrect import Speller
    # check = Speller(lang='en')
    print('1. ' + str(timeit.default_timer() - start_time))

    import nltk
    # list of english words, used to later remove non-english words
    words = set(nltk.corpus.words.words())

    # nltk.download('punkt')
    from nltk.corpus import stopwords
    # nltk.download('stopwords')
    from nltk.tokenize import word_tokenize

    # Lemmatize the words ( preferred relative to stemming )
    from nltk.stem import WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()
    # Tokenize the respective tweets
    tweet_tokens = [word_tokenize(e) for e in tagg_tweets['Text']]
    print('3. ' + str(timeit.default_timer() - start_time))
    for i in range(0,len(tweet_tokens)):
        for p in ['a','s','r','n','v']:
            tweet_tokens[i] = [lemmatizer.lemmatize(w, pos=p) for w in tweet_tokens[i]] 
            # join list of words back to a sentence    
        # result.loc[i, 'Text'] = ' '.join(tweet_tokens[i])
        # Delete all words which are non-english to reduce dimensionality of matrix later on
        tagg_tweets.loc[(i+sum_shift), 'Text'] = [" ".join(w for w in nltk.wordpunct_tokenize(str(tweet_tokens[i])) if w in words or not w.isalpha())]
    print('4. ' + str(timeit.default_timer() - start_time))

    # Negation handling:
    # Detects negations and transforms negated words into 'not_' form
    def negate_sequence(text):
        negation = False
        delims = "?.,!:;"
        result = []
        words = text.split()
        prev = None
        pprev = None
        for word in words:
            stripped = word.strip(delims).lower()
            negated = "not_" + stripped if negation else stripped
            result.append(negated)
            prev = negated

            if any(neg in word for neg in [ 'barely', 'cannot',  'hardly', 'lack', 'lacking', 'lacks', 'neither', 'never', \
                                           'no one', 'nobody', 'none',  'nothing', 'nowhere',  'scarcely', \
                                           'without', ' not ', "n't", " no "]):
                negation = not negation

            if any(c in word for c in delims):
                negation = False

        return result
    # Application of def negate_sequence for negation handling
    tagg_tweets_temp = tagg_tweets['Text'].apply(negate_sequence)            
    tagg_tweets["Text"] = list(" ".join(e) for e in tagg_tweets_temp)
    print('5. ' + str(timeit.default_timer() - start_time))
    # Remove stopwords
    # Tokenize the respective tweets
    tweet_tokens = [word_tokenize(e) for e in tagg_tweets['Text']]
    print('6. ' + str(timeit.default_timer() - start_time))
    # Define the list of stopwords which will be removed
    all_stopwords = stopwords.words('english')
    #remove_list = []
    # all_stopwords = [word for word in all_stopwords if word not in remove_list]
    # Add additional stop words to be removed from texts
    sw_list = ['amp', 'aint', "ain't", 'barely', 'cannot ', 'cant', "can't", 'couldnt', "couldn't", 'darent', "daren't", 'didnt', \
    "didn't", "doesnt", "doesn't", 'dont', "don't", 'hadnt', "hadn't", 'hardly', 'hasnt', "hasn't", 'havent', \
    "haven't", 'isnt', "isn't", 'lack', 'lacking', 'lacks', 'mightnt', "mightn't", 'mustnt', "mustn't", \
    'neednt', "needn't", 'neither', 'never', 'no one', 'nobody', 'none', 'nor',  'nothing', \
    'nowhere', 'oughtnt', "oughtn't", 'scarcely', 'shant', "shan't", 'shouldnt', "shouldn't", 'wasnt', \
    "wasn't", 'without', 'wont', "won't", 'wouldnt', "wouldn't"]
    all_stopwords.extend(sw_list)
#        remove_list = ['not']
#        all_stopwords = [word for word in all_stopwords if word not in remove_list]
    for i in range(0,len(tweet_tokens)):        
        # Delete all stopwords
        tagg_tweets.loc[(i+sum_shift), 'Text'] = [" ".join(word for word in tweet_tokens[i] if word not in all_stopwords)]        


    # Remove all punctuations and all special chars and replace with space
    import string
    tagg_tweets['Text'] = tagg_tweets['Text'].str.replace('[^\w\s]',' ')

    # Remove all numbers
    tagg_tweets['Text'] = [re.sub("\d+", "", e) for e in tagg_tweets['Text']]

    # Remove short words with 2 chars or less
    tagg_tweets['Text'] = [re.sub(r'\b\w{1,2}\b', '', e) for e in tagg_tweets['Text']]

    # Replace multiple whitespaces with single whitespace
    tagg_tweets['Text'] = [' '.join(str(e).split()) for e in tagg_tweets['Text']]
    print('7. ' + str(timeit.default_timer() - start_time))

    return tagg_tweets

tagg_tweets = list(map(large_pre_proc,list(range(0, (N_splits)))))

# Merge file
new_tagg_tweets = pd.DataFrame(tagg_tweets[0])
for x in range (1,N_splits):
    new_tagg_tweets = new_tagg_tweets.append(pd.DataFrame(tagg_tweets[x]), ignore_index=True) 
new_tagg_tweets = new_tagg_tweets[['Sentiment','Text']]
new_tagg_tweets.to_csv(r'C:\Users\muell\Desktop\Thesis\MA\Data_Tagged\training_1600000_clean.csv')



0. 7.459310699996422
1. 8.0694033999971
3. 15.538733900000807
4. 54.21997209999972
5. 56.32498209999903
6. 61.11991829999897
7. 84.62575239999569
0. 84.6758417999954
1. 84.9717845999985
3. 88.38962049999827
4. 113.75850910000008
5. 116.27590280000004
6. 121.1004352999953
7. 144.20949459999974
0. 144.25879829999758
1. 144.53136829999858
3. 148.1999066999997
4. 174.07531200000085
5. 176.120917799999
6. 181.46306470000127
7. 204.74822599999607
0. 204.7980133000019
1. 205.076527600002
3. 208.40280319999874
4. 234.19747110000026
5. 236.36134080000193
6. 241.35615400000097
7. 265.730974099999
0. 265.78226449999784
1. 266.0647847
3. 269.46988549999514
4. 296.0691097000017
5. 298.2642524999974
6. 303.09060099999624
7. 326.9310323999962
0. 326.9835979999989
1. 327.2572607999973
3. 330.7683784999972
4. 356.2809283999959
5. 358.37681449999945
6. 363.1714292999968
7. 385.9909817000007
0. 386.0403567999965
1. 386.31663639999897
3. 389.8802280000018
4. 416.2317363999973
5. 418.3514765
6. 423.6627364

1. 3507.915203999997
3. 3511.497409399999
4. 3535.589625200002
5. 3537.5789914999987
6. 3542.453275699998
7. 3564.551000499996
0. 3564.595456999996
1. 3564.8645349
3. 3568.2096644999983
4. 3593.0025793999957
5. 3594.872773999996
6. 3599.580565999997
7. 3621.8464823999966
0. 3621.890795899999
1. 3622.1591298999992
3. 3625.6901509999952
4. 3649.9896051999967
5. 3652.1246209999954
6. 3657.2277004999996
7. 3679.3372363000017
0. 3679.382724999996
1. 3679.655423199998
3. 3682.979513500002
4. 3707.4961781999955
5. 3709.4439646999963
6. 3714.327894199996
7. 3736.8464760999996
0. 3736.8911725
1. 3737.157319699996
3. 3740.4528417999973
4. 3766.773208300001
5. 3769.5699892999983
6. 3774.6741755999974
7. 3798.169173000002
0. 3798.213574100002
1. 3798.4918158
3. 3801.9315408999973
4. 3826.0784617000027
5. 3828.1807235000015
6. 3833.1076933000004
7. 3856.1051675000053
0. 3856.149550300004
1. 3856.426139599993
3. 3859.881185899998
4. 3885.0342935999943
5. 3887.0458877999918
6. 3891.8525467
7. 3914.34

### Select most sentiment-rich words and allocate pos/neg sentiment 

In [ ]:
# Read-in saved csv
tagg_tweets = pd.read_csv(r'C:\Users\muell\Desktop\Thesis\MA\Data_Tagged\training_1600000_clean.csv')
# Kick out all unrequired columns (existed in testing phase of code)
tagg_tweets = tagg_tweets[['Sentiment','Text']]
# Fill cells without any string with na
tagg_tweets['Text'] = tagg_tweets['Text'].fillna('')
# Remove all tweets with negations (i.e. 'not_'). Avoids wrong handling of negations and results in sufficient df with 
# 1'428'989 Observations/rows/tweets (out of 1.6M)
tagg_tweets = tagg_tweets[~tagg_tweets.Text.str.contains('not_')]
# Split dataset into negative and positive, such that memory of laptop can handle subsequent operations
tagg_tweets_neg = tagg_tweets[tagg_tweets['Sentiment'] == 0]
tagg_tweets_pos = tagg_tweets[tagg_tweets['Sentiment'] == 1]
# Apply TF-IDF weighting to select 2000 most highly weighted features per sentiment pole (i.e. max_features=1000), with 
# minimum of 100 occurences (i.e. min_df=100) and maximum occurence in 15% of tweets (i.e. max_df=0.15)
from sklearn.feature_extraction.text import TfidfVectorizer  
from nltk.corpus import stopwords
vectorizer = TfidfVectorizer(max_features=2000, min_df=100, max_df=0.15, stop_words=stopwords.words('english'))  
# Apply tf-idf weighting to positive sample
tf_idf_pos = vectorizer.fit_transform(tagg_tweets_pos.Text)
# Extract the 1000 feature names from prior tf-idf weighting
feature_names_pos['Features']  = pd.DataFrame(np.array(vectorizer.get_feature_names()))
# Same for negative sentiment
tf_idf_neg = vectorizer.fit_transform(tagg_tweets_neg.Text)
feature_names_neg['Features'] = pd.DataFrame(np.array(vectorizer.get_feature_names()))
# Create one df with all 1k negative + positive features and remove duplicates
feature_names = feature_names_pos.head(n=1000).append(feature_names_neg.head(n=1000)).drop_duplicates()
# Kick out all unrequired columns (existed in testing phase of code)
feature_names = feature_names[['Features']]
# Count all positive occurences of each word on the feature_names list
count_pos = \
    pd.Series({w: tagg_tweets_pos['Text'].str.contains(w, case=False).sum() for w in list(feature_names['Features'])})
# Store the resulting series in a df
count_pos_df = pd.DataFrame({'Features':count_pos.index, 'Count_pos':count_pos.values})
# Left join the count of positive occurences to the main df feature_names
feature_names = pd.merge(feature_names,count_pos_df,on='Features',how='left')
# Same for negative word occurences
count_neg = \
    pd.Series({w: tagg_tweets_neg['Text'].str.contains(w, case=False).sum() for w in list(feature_names['Features'])})
count_neg_df = pd.DataFrame({'Features':count_neg.index, 'Count_neg':count_neg.values})
feature_names = pd.merge(feature_names,count_neg_df,on='Features',how='left')
# Create column with percentage of positive vs overall occurences of the respective word
feature_names['Perc_pos'] = feature_names['Count_pos']/(feature_names['Count_pos'] + feature_names['Count_neg'])
# Create column with respective sentiment, default is 0
feature_names['Sentiment'] = 0
# If 75% or more of occurences are in positive context than word is attributed positive sentiment
feature_names.loc[feature_names['Perc_pos'] >= 0.75, 'Sentiment'] = 1
# Same for negative
feature_names.loc[feature_names['Perc_pos'] <= 0.25, 'Sentiment'] = -1
# Drop all words with neutral sentiment (i.e. feature_names['Sentiment'] == 0)
feature_names = feature_names[~(feature_names['Sentiment'] == 0)]
# write to csv to join in excel with other lexica and read-in later on
feature_names.to_csv(r'C:\Users\muell\Desktop\Thesis\MA\Data_Tagged\feature_names.csv')


In [117]:
import theano

### Ad-hoc analysis tries (re-use might be advantageous)

In [ ]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt

text = ' '.join(word for word in result.Text)
print ("There are {} words in the combination of all review.".format(len(text)))


# Create and generate a word cloud image:
wordcloud = WordCloud().generate(text)

# Display the generated image:
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [84]:
Dummy = 1
if Dummy == 1:
    stock_adj = "AAPL_adj"
    conn = sqlite3.connect(r"C:\Users\muell\Desktop\Thesis\MA\SQL_DB\Tw_DB.db")
    c = conn.cursor()
    # Select entire table of respective stock ticker
    query = '''select ID, Text from {} '''.format(stock_adj)
    c.execute(query)
    # Convert to pandas DF format
    result = pd.DataFrame(c.fetchall())
    # Rename all columns accordingly
    result.rename(columns={0: 'ID', 1: 'Text'}, inplace=True)

else:
    start_time = timeit.default_timer()
    for a in range(0,len(stocklist)):

        stock_adj = stocklist[a] + '_adj'
        stock = stocklist[a]
        conn = sqlite3.connect(r"C:\Users\muell\Desktop\Thesis\MA\SQL_DB\Tw_DB.db")
        c = conn.cursor()
        # Select entire table of respective stock ticker
        query = '''select ID, Text from {} '''.format(stock_adj)
        c.execute(query)
        # Convert to pandas DF format
        if a == 0:
            result = pd.DataFrame(c.fetchall())
            # Rename all columns accordingly
            result.rename(columns={0: 'ID', 1: 'Text'}, inplace=True)
        else: 
            result_add = pd.DataFrame(c.fetchall())
            # Rename all columns accordingly
            result_add.rename(columns={0: 'ID', 1: 'Text'}, inplace=True)
            result = pd.concat([result, result_add])
    print(result)  
    print('Final: ' + str(timeit.default_timer() - start_time))



In [90]:
start_time = timeit.default_timer()
result_rand = result.sample(frac=1)
# Enough memory for 800k x 2k tf-idf mat
# len_samp = round(len(result_rand)/4)
# print(len_samp)
# result_rand_1 = result_rand[:len_samp]
# result_rand_2 = result_rand[len_samp:]
print('1: ' + str(timeit.default_timer() - start_time))


from sklearn.feature_extraction.text import TfidfVectorizer  
vectorizer = TfidfVectorizer(max_features=2000, min_df=100, max_df=0.15, stop_words=stopwords.words('english'))  
tf_idf_1 = pd.DataFrame(vectorizer.fit_transform(result_rand_1.Text).toarray())
print(tf_idf_1)
    
feature_names = pd.DataFrame(np.array(vectorizer.get_feature_names()))
print(feature_names)
print('Final: ' + str(timeit.default_timer() - start_time))


320768
1: 0.003376300010131672


MemoryError: Unable to allocate 17.9 GiB for an array with shape (800000, 3000) and data type float64